In [1]:
# ! pip install numpy pandas jupyter notebook tqdm

In [16]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd

DATA_DIR = Path("./datasets/")

# Download dataset

In [19]:
# ! kaggle datasets download ipythonx/ade20k-scene-parsing

In [20]:
# ! mkdir datasets && \
#     mv ade20k-scene-parsing.zip ./datasets && \
#     cd datasets && unzip ade20k-scene-parsing.zip

# Download SPADE 

In [17]:
# Uncomment command bellow if running first time!

# ! git clone https://github.com/NVlabs/SPADE.git && \
#     cd SPADE/ && \
#     pip install -r requirements.txt && \
#     cd models/networks/ && \
#     git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch && \
#     cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .

# Prepare for training

In [18]:
df = pd.read_csv(DATA_DIR / "ADEChallengeData2016/sceneCategories.txt", sep=" ", header=None)
df

,0,1
0,ADE_train_00000001,airport_terminal
1,ADE_train_00000002,airport_terminal
2,ADE_train_00000003,art_gallery
3,ADE_train_00000004,badlands
4,ADE_train_00000005,ball_pit
...,...,...
22205,ADE_val_00001996,swimming_hole
22206,ADE_val_00001997,valley
22207,ADE_val_00001998,block
22208,ADE_val_00001999,fan


In [21]:
# train_dir = DATA_DIR / 'ADE20K_2016_07_26/images/training'
# val_dir = DATA_DIR / 'ADE20K_2016_07_26/images/validation'
# classes = ['bedroom', 'childs_room', 'dining_room', 'hallway', 'kitchen', 'living_room']

# out_train_dir = DATA_DIR / "indoor_data/images/training"
# out_val_dir = DATA_DIR / "indoor_data/images/validation"

# for src, dst in [[train_dir, out_train_dir], [val_dir, out_val_dir]]:
#     dst.mkdir(exist_ok=True, parents=True)

#     for label in tqdm(classes):
#         label_dir = dst / label
#         label_dir.mkdir()
#         for file in (src / label[0] / label).iterdir():
#             (label_dir / label[0]).mkdir(exist_ok=True)
#             (label_dir / label[0] / file.name).write_bytes(file.read_bytes())



src = DATA_DIR / "ADEChallengeData2016"
dst = DATA_DIR / "indoor_data"
classes = ['bedroom', 'childs_room', 'dining_room', 'hallway', 'kitchen', 'living_room']

added_classes = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    filename, cls = row[0], row[1]
    if cls in classes:
        if cls not in added_classes:
            added_classes.append(cls)
        
        if "train" in filename:
            src_img = src / "images/training" / f"{filename}.jpg"
            src_ann = src / "annotations/training" / f"{filename}.png"
        else:
            src_img = src / "images/validation" / f"{filename}.jpg"
            src_ann = src / "annotations/validation" / f"{filename}.png"

        (dst / "images").mkdir(exist_ok=True, parents=True)
        (dst / "annotations").mkdir(exist_ok=True, parents=True)

        (dst / "images" / f"{filename}.jpg").write_bytes(src_img.read_bytes())
        (dst / "annotations" / f"{filename}.png").write_bytes(src_ann.read_bytes())

100%|██████████| 22210/22210 [00:02<00:00, 8974.65it/s] 


In [24]:
len(added_classes)

6

# Train

In [4]:
# ! pip install -U torchvision==0.11.2 torch==1.10.2 --extra-index-url https://download.pytorch.org/whl/cu113
# ! pip install -U tensorflow==1.15 scipy==1.2

In [25]:
! PYTHONPATH=$PWD/SPADE python SPADE/train.py -h

usage: train.py [-h] [--name NAME] [--gpu_ids GPU_IDS]
                [--checkpoints_dir CHECKPOINTS_DIR] [--model MODEL]
                [--norm_G NORM_G] [--norm_D NORM_D] [--norm_E NORM_E]
                [--phase PHASE] [--batchSize BATCHSIZE]
                [--preprocess_mode {resize_and_crop,crop,scale_width,scale_width_and_crop,scale_shortside,scale_shortside_and_crop,fixed,none}]
                [--load_size LOAD_SIZE] [--crop_size CROP_SIZE]
                [--aspect_ratio ASPECT_RATIO] [--label_nc LABEL_NC]
                [--contain_dontcare_label] [--output_nc OUTPUT_NC]
                [--dataroot DATAROOT] [--dataset_mode DATASET_MODE]
                [--serial_batches] [--no_flip] [--nThreads NTHREADS]
                [--max_dataset_size MAX_DATASET_SIZE] [--load_from_opt_file]
                [--cache_filelist_write] [--cache_filelist_read]
                [--display_winsize DISPLAY_WINSIZE] [--netG NETG] [--ngf NGF]
                [--init_type INIT_TYPE] [--init_var

In [28]:
! PYTHONPATH=$PWD/SPADE python SPADE/train.py \
    --name room_seg \
    --model pix2pix \
    --phase train \
    --batchSize 4 \
    --dataroot '/home/and/projects/itmo/itmo_image_rec_gen/04_segmentation_to_image/datasets/indoor_data' \
    --dataset_mode 'ade20k' \
    --nThreads 4 \
    --tf_log \
    --niter 50 \
    --niter_decay 30 \
    --optimizer adam \
    --continue_train

----------------- Options ---------------
            D_steps_per_G: 1                             
             aspect_ratio: 1.0                           
                batchSize: 4                             	[default: 1]
                    beta1: 0.0                           
                    beta2: 0.9                           
      cache_filelist_read: False                         
     cache_filelist_write: False                         
          checkpoints_dir: ./checkpoints                 
   contain_dontcare_label: True                          
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /home/and/projects/itmo/itmo_image_rec_gen/04_segmentation_to_image/datasets/indoor_data	[default: ./datasets/cityscapes/]
             dataset_mode: ade20k                        	[default: coco]
                    debug: False                         
        

# Inference

In [52]:
! PYTHONPATH=$PWD/SPADE CUDA_LAUNCH_BLOCKING=1 python SPADE/test.py \
    --name room_seg \
    --dataset_mode ade20k \
    --dataroot '/home/and/projects/itmo/itmo_image_rec_gen/04_segmentation_to_image/datasets/demo' \
    --phase test \
    --which_epoch 80

----------------- Options ---------------
             aspect_ratio: 1.0                           
                batchSize: 1                             
      cache_filelist_read: False                         
     cache_filelist_write: False                         
          checkpoints_dir: ./checkpoints                 
   contain_dontcare_label: True                          
                crop_size: 256                           
                 dataroot: /home/and/projects/itmo/itmo_image_rec_gen/04_segmentation_to_image/datasets/demo	[default: ./datasets/cityscapes/]
             dataset_mode: ade20k                        	[default: coco]
          display_winsize: 256                           
                  gpu_ids: 0                             
                 how_many: inf                           
                init_type: xavier                        
            init_variance: 0.02                          
                  isTrain: False             